## 导入相关的包

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses, optimizers, datasets
from tensorflow.keras.callbacks import TensorBoard

## 加载数据集并进行预处理

In [2]:
def preprocess(x, y):
    """
    预处理函数
    """
    x = tf.cast(x, dtype=tf.float32) / 255
    x = tf.expand_dims(x, axis=3)
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x,y

In [3]:
# 加载手写数据集
(x, y), (x_test, y_test) = datasets.mnist.load_data()

print(x.shape, y.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
batchsz = 1000


# 转化为Dataset数据集
train_db = tf.data.Dataset.from_tensor_slices((x, y))

# 随机打散
train_db = train_db.shuffle(10000)

train_db = train_db.batch(batchsz)

# 数据预处理
train_db = train_db.map(preprocess)

In [5]:
sample = sample = next(iter(train_db))
print('batch:', sample[0].shape, sample[1].shape)
print('batch:', sample[0].shape, sample[1].shape)

batch: (1000, 28, 28, 1) (1000, 10)
batch: (1000, 28, 28, 1) (1000, 10)


In [6]:
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.batch(batchsz).map(preprocess)

## 创建网络层

In [7]:
model = Sequential([
    layers.Conv2D(6, kernel_size=5, strides=1, activation="relu"), # Conv Layer 1
    layers.MaxPool2D(pool_size=2, strides=2), # Pooling Layer 2
    layers.Conv2D(16, kernel_size=5, strides=1, activation="relu"), # Conv Layer 3
    layers.MaxPool2D(pool_size=2, strides=2), # Pooling Layer 4
    layers.Flatten(), # flatten 层，方便全连接处理
    layers.Dense(120, activation="relu"), # Fully connected layer 1
    layers.Dense(84, activation="relu"), # Fully connected layer 2
    layers.Dense(10) # Fully connected layer 
])

## 打印网络信息

In [8]:
model.build(input_shape=(None, 28, 28, 1))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  30840     
_________________________________________________________________
dense_1 (Dense)              multiple                  1

## 模型训练

In [10]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-3),
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [11]:
model.fit(
    train_db,
    epochs=5,
    validation_data=test_db,
    validation_freq=2
)

Train for 60 steps, validate for 10 steps
Epoch 1/5
60/60 [==============================] - 15s 253ms/step - loss: 1.0019 - accuracy: 0.7417
Epoch 2/5
60/60 [==============================] - 15s 246ms/step - loss: 0.2996 - accuracy: 0.9113 - val_loss: 0.2260 - val_accuracy: 0.9320
Epoch 3/5
60/60 [==============================] - 14s 232ms/step - loss: 0.2050 - accuracy: 0.9399
Epoch 4/5
60/60 [==============================] - 15s 243ms/step - loss: 0.1517 - accuracy: 0.9545 - val_loss: 0.1196 - val_accuracy: 0.9625
Epoch 5/5
60/60 [==============================] - 14s 228ms/step - loss: 0.1231 - accuracy: 0.9631


## 模型验证

In [12]:
model.evaluate(test_db)

10/10 [==============================] - 1s 71ms/step - loss: 0.0971 - accuracy: 0.9701


[0.09712852500379085, 0.9701]